# Analysis of County-Level US Presidential Election Winners

## Overview
I was tasked with creating a model to predict which party will win more votes in a county in a United States presidential election based on various demographic features of that county. This model can be used by political parties to identify emerging counties of interest in the lead-up to an election to help guide their decisions on where to allocate advertising, field organizing, and other resources.

I used county-level demographic and presidential election data from the US Census Bureau and MIT Election Data + Science Lab that had been consolidated into a single dataset on Kaggle. The data included information from four presidential elections (2008, 2012, 2016, and 2020), each of which took place in a different national political context and with varying levels of national election turnout.

[MODEL DESCRIPTION]

[CONCLUSION]

## Business Problem
Despite winning three of the last four presidential elections, the Democratic Party in the United States faces a perilous political future. A [small number of states](https://www.cnn.com/2022/11/22/politics/2022-preview-2024-presidential-election/index.html) hold the key to swinging any given presidential contest. The near-inverse results of the 2016 and 2020 elections in terms of total electoral college votes despite [small swings in the popular vote margins of these states](https://www.washingtonpost.com/elections/election-results/president-2020/) demonstrates the importance of allocating campaign resources efficiently.

The 2020 election proved that the Democratic Party can return states it had lost in 2016 to its column (Michigan, Wisconsin, Pennsylvania) while simultaneously picking up new states for the first time since the 1990s (Arizona, Georgia). In order to protect these gains, and expand the electoral map to open new opportunities, the party must have an understanding of what factors make a state likely to tip to one party or the other in the coming years.

I chose to conduct my analysis at the county level because while presidential elections are decided at the state level, political parties build operations (such as "get out the vote" efforts and digital advertising campaigns) that target voters at the most granular geographic level possible and need information to guide these decisions. Additionally, these predictions are only meant to be one piece of the puzzle - they can be put in the broader political context of the state and nation as part of the process of developing a comprehensive campaign strategy. 

By building a predictive model to project the winner of presidential elections at the county level, the party can better understand what factors make a given area likely to vote one way or the other in coming elections. It can use this knowledge to guide its investments in emerging fertile areas or to shore up states that are at risk of slipping away.

In [88]:
# Standard Packages
import pandas as pd
import numpy as np

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt

# Scipy Stats
import scipy.stats as stats

# SKLearn Modules
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.metrics import r2_score, mean_absolute_error, log_loss, confusion_matrix, plot_roc_curve, classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score

# Suppress future, deprecation, and SettingWithCopy warnings
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.options.mode.chained_assignment = None

# make all columns in a df viewable
pd.options.display.max_columns = None
pd.options.display.width = None

# Data

I used [data from the US Census Bureau and MIT Election Data + Science Lab](https://www.kaggle.com/datasets/minhbtnguyen/us-census-for-election-predictions-20002020) that had been consolidated into a single dataset on Kaggle. The data included information from four presidential elections (2008, 2012, 2016, and 2020), each of which took place in a different national political context and with varying levels of national election turnout. The dataset also includes a variety of descriptive demographic statistics such as average income, average age, gender, race, education attainment, and other factors [associated with voting behavior.](https://www.icpsr.umich.edu/web/pages/instructors/setups/voting.html)

In [89]:
# load the dataset
census_data = pd.read_csv('county_census_and_election_result.csv')
census_data

,year,county_fips,inctot,mortamt1,avrg_age,ftotinc,foodstmp_1_freq,foodstmp_2_freq,sex_2_freq,sex_1_freq,marst_5_freq,marst_6_freq,marst_1_freq,marst_4_freq,marst_3_freq,marst_2_freq,race_1_freq,race_2_freq,race_7_freq,race_8_freq,race_5_freq,race_6_freq,race_3_freq,race_4_freq,race_9_freq,ctz_stat_1_freq,ctz_stat_3_freq,ctz_stat_2_freq,lang_1_freq,lang_2_freq,educ_attain_2.0_freq,educ_attain_1.0_freq,educ_attain_3.0_freq,educ_attain_4.0_freq,empstat_1.0_freq,empstat_3.0_freq,empstat_2.0_freq,state_po,county_name,democrat,green,liberitarian,other,republican,winner
0,2000,1,24566.4,634.6,46.0,46912.7,93.6,6.4,54.0,46.0,8.9,16.8,59.1,11.5,2.3,1.3,77.1,20.0,0.6,1.0,0.1,0.7,0.4,0.0,0.0,97.3,1.5,1.1,96.5,3.5,41.0,51.9,6.4,0.7,58.4,38.4,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2,33842.9,1080.1,42.4,65021.9,95.3,4.7,50.3,49.7,4.1,18.0,61.1,13.4,1.7,1.6,80.5,2.7,0.6,2.5,0.3,3.6,9.1,0.4,0.2,92.6,2.8,4.6,89.4,10.6,53.1,36.5,8.9,1.5,70.0,25.2,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,4,28331.7,814.7,45.3,52826.7,95.8,4.2,52.1,47.9,6.7,20.8,57.1,11.5,2.0,1.8,84.7,2.4,6.2,1.7,0.2,1.4,2.6,0.7,0.0,86.8,8.3,4.8,79.4,20.6,48.4,43.3,7.5,0.8,58.4,38.3,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,5,22782.6,557.5,46.2,43941.3,92.5,7.5,53.0,47.0,9.5,15.6,60.6,10.9,2.2,1.1,85.7,10.8,0.8,1.2,0.1,0.5,0.7,0.1,0.0,98.0,1.0,1.1,96.8,3.2,38.3,55.7,5.2,0.8,59.1,37.8,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,6,32245.0,1216.3,43.8,61455.3,95.7,4.3,52.0,48.0,6.4,24.5,53.7,10.6,2.4,2.5,70.5,5.4,7.9,2.6,1.3,7.8,0.9,3.5,0.2,70.9,15.2,13.9,65.6,34.4,50.2,40.1,8.5,1.2,61.4,35.1,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7521,2020,55117,26884.0,704.0,50.3,52592.7,95.6,4.4,49.5,50.5,7.3,24.2,57.1,9.4,0.9,1.1,89.7,3.1,1.1,3.4,0.3,1.9,0.1,0.1,0.2,96.4,1.7,1.8,94.4,5.6,50.9,40.7,7.8,0.6,56.3,42.3,1.4,WI,SHEBOYGAN,27101.0,2.0,896.0,403.0,37609.0,1.0
7522,2020,55127,28065.5,846.0,49.0,63315.2,94.9,5.1,49.4,50.6,5.9,26.8,56.6,8.1,0.6,2.0,91.5,0.6,3.3,2.7,0.0,0.1,0.9,0.8,0.0,93.2,2.4,4.4,91.4,8.6,56.1,33.2,9.1,1.7,61.6,37.6,0.8,WI,WALWORTH,22789.0,15.0,641.0,304.0,33851.0,1.0
7523,2020,55133,38557.0,990.6,50.4,80433.1,95.9,4.1,52.8,47.2,7.4,20.0,61.5,9.4,0.3,1.4,91.3,1.4,1.2,3.3,0.1,2.3,0.2,0.2,0.1,94.3,2.1,3.5,93.1,6.9,59.8,26.5,12.9,0.8,60.7,37.3,2.0,WI,WAUKESHA,103906.0,54.0,3023.0,1364.0,159649.0,1.0
7524,2020,55139,25796.1,667.3,45.9,49219.4,93.5,6.5,47.9,52.1,6.7,28.2,50.3,12.4,1.0,1.5,89.7,3.9,0.5,3.8,0.0,1.6,0.3,0.1,0.2,97.7,0.7,1.6,96.1,3.9,51.1,41.3,7.2,0.4,56.9,41.5,1.6,WI,WINNEBAGO,44060.0,38.0,1629.0,509.0,47796.0,1.0


In [90]:
# initial look at data
census_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7526 entries, 0 to 7525
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  7526 non-null   int64  
 1   county_fips           7526 non-null   int64  
 2   inctot                7526 non-null   float64
 3   mortamt1              7526 non-null   float64
 4   avrg_age              7526 non-null   float64
 5   ftotinc               7526 non-null   float64
 6   foodstmp_1_freq       7526 non-null   float64
 7   foodstmp_2_freq       7526 non-null   float64
 8   sex_2_freq            7526 non-null   float64
 9   sex_1_freq            7526 non-null   float64
 10  marst_5_freq          7526 non-null   float64
 11  marst_6_freq          7526 non-null   float64
 12  marst_1_freq          7526 non-null   float64
 13  marst_4_freq          7526 non-null   float64
 14  marst_3_freq          7526 non-null   float64
 15  marst_2_freq         

In [91]:
# initial look at descriptive stats of numeric values
census_data.describe()

,year,county_fips,inctot,mortamt1,avrg_age,ftotinc,foodstmp_1_freq,foodstmp_2_freq,sex_2_freq,sex_1_freq,marst_5_freq,marst_6_freq,marst_1_freq,marst_4_freq,marst_3_freq,marst_2_freq,race_1_freq,race_2_freq,race_7_freq,race_8_freq,race_5_freq,race_6_freq,race_3_freq,race_4_freq,race_9_freq,ctz_stat_1_freq,ctz_stat_3_freq,ctz_stat_2_freq,lang_1_freq,lang_2_freq,educ_attain_2.0_freq,educ_attain_1.0_freq,educ_attain_3.0_freq,educ_attain_4.0_freq,empstat_1.0_freq,empstat_3.0_freq,empstat_2.0_freq,democrat,green,liberitarian,other,republican,winner
count,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,7526.000000,1.670000e+03,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000
mean,2012.369253,27810.761228,28325.447077,884.343848,46.748220,57025.384999,90.560404,9.439596,51.956683,48.043317,6.880880,24.536606,53.739596,11.112437,1.772256,1.957720,80.788759,9.406551,2.728966,1.990194,0.265334,2.807786,0.974409,0.833285,0.208411,89.553734,5.097024,5.349641,85.797250,14.202750,48.947263,40.487164,9.304026,1.261175,58.670648,37.460218,3.869280,1.158984e+05,167.460479,668.785629,4803.562275,8.568574e+04,0.488623
std,4.916645,16197.855610,6973.567827,284.898492,2.641396,13720.562319,4.974988,4.974988,1.708150,1.708106,1.519885,6.726603,7.220689,1.966797,0.782685,0.772832,13.860190,10.505817,3.678845,2.287334,1.323770,3.698773,3.112997,1.698069,0.639464,9.207069,4.557111,5.120796,12.934326,12.934326,6.093376,9.266526,4.175560,0.919959,6.223587,5.988654,1.557713,1.918876e+05,805.370013,2083.527700,10236.284021,9.659596e+04,0.500020
min,2000.000000,1.000000,14165.200000,397.100000,36.100000,31193.400000,58.000000,0.400000,33.400000,38.400000,2.400000,10.100000,25.700000,5.300000,0.000000,0.200000,18.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.400000,0.000000,0.000000,5.700000,0.900000,25.600000,11.100000,2.000000,0.000000,30.700000,20.300000,0.300000,2.474000e+03,0.000000,0.000000,0.000000,2.187000e+03,0.000000
25%,2008.000000,13021.000000,23549.800000,680.125000,45.200000,47305.025000,87.700000,5.800000,51.100000,47.000000,5.800000,19.800000,49.400000,9.700000,1.200000,1.500000,74.400000,2.200000,0.700000,1.000000,0.000000,0.900000,0.200000,0.100000,0.000000,87.000000,2.000000,2.000000,82.300000,5.900000,45.000000,34.025000,6.300000,0.700000,54.900000,33.400000,2.700000,2.843275e+04,0.000000,0.000000,756.250000,3.562700e+04,0.000000
50%,2013.000000,28000.000000,26843.600000,808.200000,46.700000,53615.350000,91.200000,8.800000,52.100000,47.900000,6.800000,23.500000,54.200000,11.100000,1.700000,1.800000,84.300000,5.700000,1.400000,1.400000,0.100000,1.700000,0.300000,0.400000,0.100000,92.600000,3.500000,3.500000,90.500000,9.500000,49.100000,40.500000,8.500000,1.000000,59.200000,36.800000,3.600000,5.305350e+04,0.000000,0.000000,1856.000000,5.448950e+04,0.000000
75%,2017.000000,42000.000000,31579.250000,1019.500000,48.200000,63757.000000,94.200000,12.300000,53.000000,48.900000,7.900000,27.900000,58.800000,12.400000,2.200000,2.300000,91.300000,12.700000,3.200000,2.100000,0.200000,3.200000,0.700000,0.800000,0.200000,95.700000,6.500000,6.700000,94.100000,17.700000,53.000000,47.000000,11.300000,1.500000,63.000000,41.000000,4.700000,1.281622e+05,0.000000,0.000000,4808.250000,1.002938e+05,1.000000
max,2020.000000,56000.000000,68229.100000,2080.900000,59.600000,137573.700000,99.600000,42.000000,61.600000,66.600000,14.700000,54.200000,76.100000,19.500000,6.600000,8.800000,99.000000,66.400000,38.700000,53.000000,23.500000,34.400000,49.400000,23.100000,14.400000,99.900000,29.600000,35.200000,99.100000,94.300000,69.300000,69.700000,

There are a lot of missing rows in some of our columns, but that makes sense since this dataset includes information from many more years than just presidential election years. 

## Data Preparation
We're only interested in rows that have both presidential election results **and** demographic information. Let's start by dropping rows from non-presidential election years (including 2000 and 2004 since this dataset doesn't include voting data from those elections) to see what kind of missingness remains.

In [92]:
# define presidential years and only include records from those years
pres_years = [2008, 2012, 2016, 2020]
pres_df = census_data[census_data['year'].isin(pres_years)]
pres_df

,year,county_fips,inctot,mortamt1,avrg_age,ftotinc,foodstmp_1_freq,foodstmp_2_freq,sex_2_freq,sex_1_freq,marst_5_freq,marst_6_freq,marst_1_freq,marst_4_freq,marst_3_freq,marst_2_freq,race_1_freq,race_2_freq,race_7_freq,race_8_freq,race_5_freq,race_6_freq,race_3_freq,race_4_freq,race_9_freq,ctz_stat_1_freq,ctz_stat_3_freq,ctz_stat_2_freq,lang_1_freq,lang_2_freq,educ_attain_2.0_freq,educ_attain_1.0_freq,educ_attain_3.0_freq,educ_attain_4.0_freq,empstat_1.0_freq,empstat_3.0_freq,empstat_2.0_freq,state_po,county_name,democrat,green,liberitarian,other,republican,winner
1528,2008,1000,22523.9,591.8,47.0,44879.5,90.6,9.4,53.4,46.6,9.5,17.4,56.4,12.4,2.6,1.8,77.0,20.0,0.6,1.0,0.1,0.5,0.6,0.1,0.1,97.0,1.9,1.0,96.4,3.6,40.4,53.3,5.9,0.5,54.9,41.6,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1529,2008,1003,27222.4,902.2,48.6,53308.6,94.2,5.8,52.5,47.5,8.3,12.1,63.4,13.2,1.3,1.6,91.1,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,97.1,1.3,1.6,97.1,2.9,51.8,39.3,8.4,0.5,54.1,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0
1530,2008,1015,20951.6,551.7,47.1,39712.5,89.1,10.9,54.8,45.2,8.7,19.5,54.5,11.5,3.0,2.8,80.5,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,97.3,1.5,1.2,95.4,4.6,41.1,52.6,5.0,1.2,54.3,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0
1531,2008,1055,24017.6,658.6,48.2,47650.0,94.2,5.8,54.6,45.4,10.5,13.8,60.9,11.7,1.5,1.6,85.5,11.6,0.0,1.6,0.0,0.7,0.2,0.1,0.2,98.6,0.9,0.5,98.0,2.0,45.3,46.6,7.9,0.2,50.3,46.8,2.9,AL,ETOWAH,13497.0,0.0,0.0,645.0,30595.0,1.0
1532,2008,1073,28109.5,812.4,47.2,54993.2,93.5,6.5,55.0,45.0,9.5,23.5,50.3,12.3,2.5,1.9,63.1,35.1,0.4,0.7,0.0,0.5,0.1,0.1,0.0,97.5,1.3,1.2,96.6,3.4,47.8,42.0,9.3,0.9,58.5,38.1,3.4,AL,JEFFERSON,166121.0,0.0,0.0,2482.0,149921.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7521,2020,55117,26884.0,704.0,50.3,52592.7,95.6,4.4,49.5,50.5,7.3,24.2,57.1,9.4,0.9,1.1,89.7,3.1,1.1,3.4,0.3,1.9,0.1,0.1,0.2,96.4,1.7,1.8,94.4,5.6,50.9,40.7,7.8,0.6,56.3,42.3,1.4,WI,SHEBOYGAN,27101.0,2.0,896.0,403.0,37609.0,1.0
7522,2020,55127,28065.5,846.0,49.0,63315.2,94.9,5.1,49.4,50.6,5.9,26.8,56.6,8.1,0.6,2.0,91.5,0.6,3.3,2.7,0.0,0.1,0.9,0.8,0.0,93.2,2.4,4.4,91.4,8.6,56.1,33.2,9.1,1.7,61.6,37.6,0.8,WI,WALWORTH,22789.0,15.0,641.0,304.0,33851.0,1.0
7523,2020,55133,38557.0,990.6,50.4,80433.1,95.9,4.1,52.8,47.2,7.4,20.0,61.5,9.4,0.3,1.4,91.3,1.4,1.2,3.3,0.1,2.3,0.2,0.2,0.1,94.3,2.1,3.5,93.1,6.9,59.8,26.5,12.9,0.8,60.7,37.3,2.0,WI,WAUKESHA,103906.0,54.0,3023.0,1364.0,159649.0,1.0
7524,2020,55139,25796.1,667.3,45.9,49219.4,93.5,6.5,47.9,52.1,6.7,28.2,50.3,12.4,1.0,1.5,89.7,3.9,0.5,3.8,0.0,1.6,0.3,0.1,0.2,97.7,0.7,1.6,96.1,3.9,51.1,41.3,7.2,0.4,56.9,41.5,1.6,WI,WINNEBAGO,44060.0,38.0,1629.0,509.0,47796.0,1.0


In [93]:
pres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1858 entries, 1528 to 7525
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  1858 non-null   int64  
 1   county_fips           1858 non-null   int64  
 2   inctot                1858 non-null   float64
 3   mortamt1              1858 non-null   float64
 4   avrg_age              1858 non-null   float64
 5   ftotinc               1858 non-null   float64
 6   foodstmp_1_freq       1858 non-null   float64
 7   foodstmp_2_freq       1858 non-null   float64
 8   sex_2_freq            1858 non-null   float64
 9   sex_1_freq            1858 non-null   float64
 10  marst_5_freq          1858 non-null   float64
 11  marst_6_freq          1858 non-null   float64
 12  marst_1_freq          1858 non-null   float64
 13  marst_4_freq          1858 non-null   float64
 14  marst_3_freq          1858 non-null   float64
 15  marst_2_freq      

Unfortunately, we still have about 200 missing rows in our election results columns. Because our analysis requires precise adherence to actual election outcomes, there is no imputation strategy that will allow us to fill in these values without substantially diluting the value of our recommendations. We will drop these rows to ensure that we have a complete, accurate set of information for each row.

In [94]:
# drop NAs and reset index
pres_df.dropna(inplace = True)
pres_df.reset_index(drop = True, inplace = True)
pres_df

,year,county_fips,inctot,mortamt1,avrg_age,ftotinc,foodstmp_1_freq,foodstmp_2_freq,sex_2_freq,sex_1_freq,marst_5_freq,marst_6_freq,marst_1_freq,marst_4_freq,marst_3_freq,marst_2_freq,race_1_freq,race_2_freq,race_7_freq,race_8_freq,race_5_freq,race_6_freq,race_3_freq,race_4_freq,race_9_freq,ctz_stat_1_freq,ctz_stat_3_freq,ctz_stat_2_freq,lang_1_freq,lang_2_freq,educ_attain_2.0_freq,educ_attain_1.0_freq,educ_attain_3.0_freq,educ_attain_4.0_freq,empstat_1.0_freq,empstat_3.0_freq,empstat_2.0_freq,state_po,county_name,democrat,green,liberitarian,other,republican,winner
0,2008,1003,27222.4,902.2,48.6,53308.6,94.2,5.8,52.5,47.5,8.3,12.1,63.4,13.2,1.3,1.6,91.1,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,97.1,1.3,1.6,97.1,2.9,51.8,39.3,8.4,0.5,54.1,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0
1,2008,1015,20951.6,551.7,47.1,39712.5,89.1,10.9,54.8,45.2,8.7,19.5,54.5,11.5,3.0,2.8,80.5,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,97.3,1.5,1.2,95.4,4.6,41.1,52.6,5.0,1.2,54.3,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0
2,2008,1055,24017.6,658.6,48.2,47650.0,94.2,5.8,54.6,45.4,10.5,13.8,60.9,11.7,1.5,1.6,85.5,11.6,0.0,1.6,0.0,0.7,0.2,0.1,0.2,98.6,0.9,0.5,98.0,2.0,45.3,46.6,7.9,0.2,50.3,46.8,2.9,AL,ETOWAH,13497.0,0.0,0.0,645.0,30595.0,1.0
3,2008,1073,28109.5,812.4,47.2,54993.2,93.5,6.5,55.0,45.0,9.5,23.5,50.3,12.3,2.5,1.9,63.1,35.1,0.4,0.7,0.0,0.5,0.1,0.1,0.0,97.5,1.3,1.2,96.6,3.4,47.8,42.0,9.3,0.9,58.5,38.1,3.4,AL,JEFFERSON,166121.0,0.0,0.0,2482.0,149921.0,0.0
4,2008,1081,24870.4,725.1,40.6,45575.0,92.4,7.6,52.1,47.9,4.9,32.1,51.2,8.3,1.7,2.0,75.2,21.0,0.3,0.6,0.1,1.7,0.3,0.8,0.2,94.4,4.1,1.5,93.2,6.8,53.3,32.2,10.8,3.7,59.8,36.3,3.8,AL,LEE,21498.0,0.0,0.0,597.0,32230.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,2020,55105,25993.3,657.3,49.1,54440.5,91.6,8.4,51.3,48.7,7.4,27.4,49.9,13.0,0.4,1.9,89.4,3.3,1.3,4.4,0.1,0.2,0.3,0.4,0.5,96.2,1.1,2.7,96.6,3.4,51.4,40.3,7.4,0.8,53.4,43.2,3.4,WI,ROCK,46658.0,14.0,1094.0,456.0,37138.0,0.0
1666,2020,55117,26884.0,704.0,50.3,52592.7,95.6,4.4,49.5,50.5,7.3,24.2,57.1,9.4,0.9,1.1,89.7,3.1,1.1,3.4,0.3,1.9,0.1,0.1,0.2,96.4,1.7,1.8,94.4,5.6,50.9,40.7,7.8,0.6,56.3,42.3,1.4,WI,SHEBOYGAN,27101.0,2.0,896.0,403.0,37609.0,1.0
1667,2020,55127,28065.5,846.0,49.0,63315.2,94.9,5.1,49.4,50.6,5.9,26.8,56.6,8.1,0.6,2.0,91.5,0.6,3.3,2.7,0.0,0.1,0.9,0.8,0.0,93.2,2.4,4.4,91.4,8.6,56.1,33.2,9.1,1.7,61.6,37.6,0.8,WI,WALWORTH,22789.0,15.0,641.0,304.0,33851.0,1.0
1668,2020,55133,38557.0,990.6,50.4,80433.1,95.9,4.1,52.8,47.2,7.4,20.0,61.5,9.4,0.3,1.4,91.3,1.4,1.2,3.3,0.1,2.3,0.2,0.2,0.1,94.3,2.1,3.5,93.1,6.9,59.8,26.5,12.9,0.8,60.7,37.3,2.0,WI,WAUKESHA,103906.0,54.0,3023.0,1364.0,159649.0,1.0


In [95]:
# inspect cleaned data info
pres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670 entries, 0 to 1669
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  1670 non-null   int64  
 1   county_fips           1670 non-null   int64  
 2   inctot                1670 non-null   float64
 3   mortamt1              1670 non-null   float64
 4   avrg_age              1670 non-null   float64
 5   ftotinc               1670 non-null   float64
 6   foodstmp_1_freq       1670 non-null   float64
 7   foodstmp_2_freq       1670 non-null   float64
 8   sex_2_freq            1670 non-null   float64
 9   sex_1_freq            1670 non-null   float64
 10  marst_5_freq          1670 non-null   float64
 11  marst_6_freq          1670 non-null   float64
 12  marst_1_freq          1670 non-null   float64
 13  marst_4_freq          1670 non-null   float64
 14  marst_3_freq          1670 non-null   float64
 15  marst_2_freq         

In [96]:
# inspect cleaned data descriptive stats
pres_df.describe()

,year,county_fips,inctot,mortamt1,avrg_age,ftotinc,foodstmp_1_freq,foodstmp_2_freq,sex_2_freq,sex_1_freq,marst_5_freq,marst_6_freq,marst_1_freq,marst_4_freq,marst_3_freq,marst_2_freq,race_1_freq,race_2_freq,race_7_freq,race_8_freq,race_5_freq,race_6_freq,race_3_freq,race_4_freq,race_9_freq,ctz_stat_1_freq,ctz_stat_3_freq,ctz_stat_2_freq,lang_1_freq,lang_2_freq,educ_attain_2.0_freq,educ_attain_1.0_freq,educ_attain_3.0_freq,educ_attain_4.0_freq,empstat_1.0_freq,empstat_3.0_freq,empstat_2.0_freq,democrat,green,liberitarian,other,republican,winner
count,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000
mean,2014.194012,28717.220958,28841.084850,906.672635,46.903772,58749.243473,90.251257,9.748743,51.855629,48.144371,6.777066,25.837186,52.495150,11.105329,1.721317,2.062275,78.738683,9.917784,2.978862,3.136048,0.235689,3.035150,0.759401,0.928862,0.272455,88.969461,5.269042,5.761198,85.102335,14.897665,49.909222,38.882335,9.860778,1.347904,58.226347,38.028922,3.745689,1.158984e+05,167.460479,668.785629,4803.562275,8.568574e+04,0.488623
std,4.410969,15595.725527,7295.609512,291.128525,2.787388,14606.621299,4.952513,4.952513,1.830548,1.830548,1.559419,7.176813,7.509596,2.044045,0.766393,0.795502,14.646107,10.614890,3.912003,3.870875,1.090427,3.688328,2.406652,1.840668,0.680919,9.457760,4.631289,5.320845,13.393902,13.393902,5.909867,9.208955,4.384230,0.979554,6.558229,6.335632,1.373302,1.918876e+05,805.370013,2083.527700,10236.284021,9.659596e+04,0.500020
min,2008.000000,1003.000000,14897.300000,421.100000,36.700000,34914.700000,65.600000,0.900000,33.400000,42.000000,2.500000,11.600000,26.800000,5.600000,0.000000,0.500000,18.600000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,44.800000,0.000000,0.000000,6.900000,1.400000,25.600000,13.000000,2.300000,0.000000,30.700000,20.800000,0.800000,2.474000e+03,0.000000,0.000000,0.000000,2.187000e+03,0.000000
25%,2012.000000,13135.000000,23894.650000,698.025000,45.200000,48357.400000,87.300000,6.100000,51.000000,47.100000,5.700000,20.900000,48.000000,9.625000,1.200000,1.500000,71.425000,2.625000,0.725000,1.100000,0.000000,1.000000,0.200000,0.200000,0.100000,86.400000,2.125000,2.200000,81.400000,6.200000,46.100000,32.300000,6.700000,0.700000,54.300000,33.700000,2.800000,2.843275e+04,0.000000,0.000000,756.250000,3.562700e+04,0.000000
50%,2016.000000,29099.000000,27351.850000,828.550000,46.900000,55458.800000,90.900000,9.100000,52.000000,48.000000,6.700000,24.900000,52.800000,11.100000,1.600000,1.950000,82.600000,6.400000,1.600000,1.700000,0.100000,1.900000,0.300000,0.400000,0.200000,92.200000,3.600000,3.900000,89.900000,10.100000,50.000000,38.800000,9.100000,1.100000,58.700000,37.500000,3.500000,5.305350e+04,0.000000,0.000000,1856.000000,5.448950e+04,0.000000
75%,2020.000000,42018.500000,32176.200000,1039.600000,48.400000,66187.150000,93.900000,12.700000,52.900000,49.000000,7.700000,29.400000,57.600000,12.400000,2.200000,2.500000,89.700000,13.300000,3.600000,3.700000,0.200000,3.600000,0.700000,0.900000,0.300000,95.300000,6.900000,7.300000,93.800000,18.600000,54.000000,45.475000,12.100000,1.700000,62.800000,41.700000,4.500000,1.281622e+05,0.000000,0.000000,4808.250000,1.002938e+05,1.000000
max,2020.000000,55139.000000,68229.100000,2064.600000,59.400000,137573.700000,99.100000,34.400000,58.000000,66.600000,14.700000,54.100000,74.800000,19.500000,5.800000,8.500000,99.000000,65.400000,38.700000,53.000000,20.100000,33.700000,42.900000,23.100000,14.400000,99.900000,28.200000,34.600000,98.600000,93.100000,68.600000,69.700

From this initial examination it appears that we will have to drop some columns to avoid issues of multicollinearity. For example, for every row, the values of "sex_2_freq" and "sex_1_freq" will sum to 100%:

In [97]:
# checking that sum of those two columns = 100 for all rows
gender_check = pres_df.loc[pres_df['sex_2_freq'] + pres_df['sex_1_freq'] == 100]
assert gender_check.shape[0] == pres_df.shape[0]

The same goes for each discrete category of demographic information included - foodstamp frequency, gender, marriage status, race, citizenship, language spoken, educational attainment, and employment. Having perfectly (or even near-perfectly) correlated variables such as these in our data will make our model less powerful by artificially inflating its variance. 

In order to avoid multicollinearity, we will drop the column with the highest mean (which means it has the highest average proportion) from each category to serve as the "baseline" for our analysis. We can also drop the year and County FIP Code columns since they will not be used in our analysis.

In [98]:
# drop columns
todrop = ['year',
          'county_fips', 
          'foodstmp_1_freq',
          'sex_2_freq',
          'marst_1_freq',
          'race_1_freq',
          'ctz_stat_1_freq',
          'lang_1_freq',
          'educ_attain_2.0_freq',
          'empstat_1.0_freq']

pres_df.drop(todrop, inplace = True, axis = 1)
pres_df.head(2)

,inctot,mortamt1,avrg_age,ftotinc,foodstmp_2_freq,sex_1_freq,marst_5_freq,marst_6_freq,marst_4_freq,marst_3_freq,marst_2_freq,race_2_freq,race_7_freq,race_8_freq,race_5_freq,race_6_freq,race_3_freq,race_4_freq,race_9_freq,ctz_stat_3_freq,ctz_stat_2_freq,lang_2_freq,educ_attain_1.0_freq,educ_attain_3.0_freq,educ_attain_4.0_freq,empstat_3.0_freq,empstat_2.0_freq,state_po,county_name,democrat,green,liberitarian,other,republican,winner
0,27222.4,902.2,48.6,53308.6,5.8,47.5,8.3,12.1,13.2,1.3,1.6,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,1.3,1.6,2.9,39.3,8.4,0.5,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0
1,20951.6,551.7,47.1,39712.5,10.9,45.2,8.7,19.5,11.5,3.0,2.8,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,1.5,1.2,4.6,52.6,5.0,1.2,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0


In [103]:
# check that they're gone
assert not any(col in pres_df.columns for col in todrop)

Now that we have a complete dataset, we need to take a look at the descriptive statistics to see if there are any obvious outliers or data entry errors. These column headers are difficult to interpret - let's replace them with their description from the metadata included with the data to make it simpler to understand.

In [104]:
# load descriptions and create a dictionary with original column names and their description
descriptions = pd.read_csv('meta_data.csv')
descriptions_dict = dict(zip(descriptions['Column'], descriptions['Description']))
descriptions_dict

{'year': 'Year',
 'county_fips': 'County FIP Code',
 'inctot': 'Average Annual Income of Individuals',
 'mortamt1': 'Average Amount Of Mortgage Payments',
 'avrg_age': 'Average Age',
 'ftotinc': 'Average Annual Total Family Income',
 'foodstmp_1_freq': '% of Individuals Who Did Not Receive Foodstamps',
 'foodstmp_2_freq': '% of Individuals Who Received Foodstamps',
 'sex_2_freq': '% of Females',
 'sex_1_freq': '% of Males',
 'marst_5_freq': '% Widowed',
 'marst_6_freq': '% Never Married',
 'marst_1_freq': '% Married Spouse Present',
 'marst_4_freq': '% Divorced',
 'marst_3_freq': '% Seperated',
 'marst_2_freq': '% Married Spouse Absent',
 'race_1_freq': '% White',
 'race_2_freq': '% Black/African American',
 'race_7_freq': '% Other Race',
 'race_8_freq': '% Two major Races',
 'race_5_freq': '% Japanese',
 'race_6_freq': '% Other Asian',
 'race_3_freq': '% American Indian or Alaska Native',
 'race_4_freq': '% Chinese',
 'race_9_freq': '% Three or More Races',
 'ctz_stat_1_freq': '% Citi

In [105]:
# rename columns using descriptions
pres_df.rename(columns=descriptions_dict, inplace=True)
pres_df.head(2)

,Average Annual Income of Individuals,Average Amount Of Mortgage Payments,Average Age,Average Annual Total Family Income,% of Individuals Who Received Foodstamps,% of Males,% Widowed,% Never Married,% Divorced,% Seperated,% Married Spouse Absent,% Black/African American,% Other Race,% Two major Races,% Japanese,% Other Asian,% American Indian or Alaska Native,% Chinese,% Three or More Races,% Non-Citizen,% Naturalized Citizen,% Another Language Is Spoken At Home,% High School or Lower Education,% Masters or Professional Certificate,% Doctoral Degree,% Not In The Labor Force,% Unemployed,state_po,county_name,democrat,green,liberitarian,other,republican,winner
0,27222.4,902.2,48.6,53308.6,5.8,47.5,8.3,12.1,13.2,1.3,1.6,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,1.3,1.6,2.9,39.3,8.4,0.5,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0
1,20951.6,551.7,47.1,39712.5,10.9,45.2,8.7,19.5,11.5,3.0,2.8,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,1.5,1.2,4.6,52.6,5.0,1.2,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0


In [106]:
pres_df.describe()

,Average Annual Income of Individuals,Average Amount Of Mortgage Payments,Average Age,Average Annual Total Family Income,% of Individuals Who Received Foodstamps,% of Males,% Widowed,% Never Married,% Divorced,% Seperated,% Married Spouse Absent,% Black/African American,% Other Race,% Two major Races,% Japanese,% Other Asian,% American Indian or Alaska Native,% Chinese,% Three or More Races,% Non-Citizen,% Naturalized Citizen,% Another Language Is Spoken At Home,% High School or Lower Education,% Masters or Professional Certificate,% Doctoral Degree,% Not In The Labor Force,% Unemployed,democrat,green,liberitarian,other,republican,winner
count,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000
mean,28841.084850,906.672635,46.903772,58749.243473,9.748743,48.144371,6.777066,25.837186,11.105329,1.721317,2.062275,9.917784,2.978862,3.136048,0.235689,3.035150,0.759401,0.928862,0.272455,5.269042,5.761198,14.897665,38.882335,9.860778,1.347904,38.028922,3.745689,1.158984e+05,167.460479,668.785629,4803.562275,8.568574e+04,0.488623
std,7295.609512,291.128525,2.787388,14606.621299,4.952513,1.830548,1.559419,7.176813,2.044045,0.766393,0.795502,10.614890,3.912003,3.870875,1.090427,3.688328,2.406652,1.840668,0.680919,4.631289,5.320845,13.393902,9.208955,4.384230,0.979554,6.335632,1.373302,1.918876e+05,805.370013,2083.527700,10236.284021,9.659596e+04,0.500020
min,14897.300000,421.100000,36.700000,34914.700000,0.900000,42.000000,2.500000,11.600000,5.600000,0.000000,0.500000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.400000,13.000000,2.300000,0.000000,20.800000,0.800000,2.474000e+03,0.000000,0.000000,0.000000,2.187000e+03,0.000000
25%,23894.650000,698.025000,45.200000,48357.400000,6.100000,47.100000,5.700000,20.900000,9.625000,1.200000,1.500000,2.625000,0.725000,1.100000,0.000000,1.000000,0.200000,0.200000,0.100000,2.125000,2.200000,6.200000,32.300000,6.700000,0.700000,33.700000,2.800000,2.843275e+04,0.000000,0.000000,756.250000,3.562700e+04,0.000000
50%,27351.850000,828.550000,46.900000,55458.800000,9.100000,48.000000,6.700000,24.900000,11.100000,1.600000,1.950000,6.400000,1.600000,1.700000,0.100000,1.900000,0.300000,0.400000,0.200000,3.600000,3.900000,10.100000,38.800000,9.100000,1.100000,37.500000,3.500000,5.305350e+04,0.000000,0.000000,1856.000000,5.448950e+04,0.000000
75%,32176.200000,1039.600000,48.400000,66187.150000,12.700000,49.000000,7.700000,29.400000,12.400000,2.200000,2.500000,13.300000,3.600000,3.700000,0.200000,3.600000,0.700000,0.900000,0.300000,6.900000,7.300000,18.600000,45.475000,12.100000,1.700000,41.700000,4.500000,1.281622e+05,0.000000,0.000000,4808.250000,1.002938e+05,1.000000
max,68229.100000,2064.600000,59.400000,137573.700000,34.400000,66.600000,14.700000,54.100000,19.500000,5.800000,8.500000,65.400000,38.700000,53.000000,20.100000,33.700000,42.900000,23.100000,14.400000,28.200000,34.600000,93.100000,69.700000,36.600000,7.500000,67.200000,10.400000,3.028885e+06,21660.000000,35452.000000,200201.000000,1.145530e+06,1.000000


From examining the head of the data and the descriptive statistics, there are a couple of conclusions we can draw:
- There are some potential outliers (such as the county where only 6.9% of people speak English at home, or the county where over 67% of the population is not in the labor force).
- These counties have a diverse set of demographic characteristics along a variety of lines of difference, such as race, marriage status, language spoken at home, and more.

Let's examine the distribution of each column to check for potential outliers:

## Feature Engineering: District Competitiveness, Census Regions

In [13]:
def calculate_percent_col(df, numerator_col, denominator_cols):
    ''' 
    A function that takes three arguments: a dataframe, a string for a column
    in the numerator, and a string or list for a column(s) in the denominator.
    
    It sums the values of the column(s) in the denominator and divides the value
    from the numerator column by that summed amount. It then multiplies that value
    by 100 to represent a percentage, and returns that value.
    '''
    denominator_sum = df[denominator_cols].sum(axis=1)
    percent_col = (df[numerator_col] / denominator_sum) * 100
    return percent_col

In [14]:
parties = ['republican', 'democrat', 'liberitarian', 'other', 'green']

In [15]:
# creating a percent republican column
pres_df['percent_rep'] = calculate_percent_col(pres_df, 'republican', parties)

# creating a percent democrat column
pres_df['percent_dem'] = calculate_percent_col(pres_df, 'democrat', parties)

In [16]:
pres_df.head(4)

,Year,County FIP Code,Average Annual Income of Individuals,Average Amount Of Mortgage Payments,Average Age,Average Annual Total Family Income,% of Individuals Who Did Not Receive Foodstamps,% of Individuals Who Received Foodstamps,% of Females,% of Males,% Widowed,% Never Married,% Married Spouse Present,% Divorced,% Seperated,% Married Spouse Absent,% White,% Black/African American,% Other Race,% Two major Races,% Japanese,% Other Asian,% American Indian or Alaska Native,% Chinese,% Three or More Races,% Citizen,% Non-Citizen,% Naturalized Citizen,% English Is Spoken At Home,% Another Language Is Spoken At Home,% Some College or Bachelor Degree,% High School or Lower Education,% Masters or Professional Certificate,% Doctoral Degree,% Employed,% Not In The Labor Force,% Unemployed,state_po,county_name,democrat,green,liberitarian,other,republican,winner,percent_rep,percent_dem
0,2008,1003,27222.4,902.2,48.6,53308.6,94.2,5.8,52.5,47.5,8.3,12.1,63.4,13.2,1.3,1.6,91.1,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,97.1,1.3,1.6,97.1,2.9,51.8,39.3,8.4,0.5,54.1,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0,75.259479,23.811922
1,2008,1015,20951.6,551.7,47.1,39712.5,89.1,10.9,54.8,45.2,8.7,19.5,54.5,11.5,3.0,2.8,80.5,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,97.3,1.5,1.2,95.4,4.6,41.1,52.6,5.0,1.2,54.3,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0,65.691889,33.170870
2,2008,1055,24017.6,658.6,48.2,47650.0,94.2,5.8,54.6,45.4,10.5,13.8,60.9,11.7,1.5,1.6,85.5,11.6,0.0,1.6,0.0,0.7,0.2,0.1,0.2,98.6,0.9,0.5,98.0,2.0,45.3,46.6,7.9,0.2,50.3,46.8,2.9,AL,ETOWAH,13497.0,0.0,0.0,645.0,30595.0,1.0,68.388582,30.169658
3,2008,1073,28109.5,812.4,47.2,54993.2,93.5,6.5,55.0,45.0,9.5,23.5,50.3,12.3,2.5,1.9,63.1,35.1,0.4,0.7,0.0,0.5,0.1,0.1,0.0,97.5,1.3,1.2,96.6,3.4,47.8,42.0,9.3,0.9,58.5,38.1,3.4,AL,JEFFERSON,166121.0,0.0,0.0,2482.0,149921.0,0.0,47.067411,52.153370


In [17]:
# creating a percent margin column
pres_df['winning_margin'] = pres_df['percent_rep'] - pres_df['percent_dem']
pres_df.head(2)

,Year,County FIP Code,Average Annual Income of Individuals,Average Amount Of Mortgage Payments,Average Age,Average Annual Total Family Income,% of Individuals Who Did Not Receive Foodstamps,% of Individuals Who Received Foodstamps,% of Females,% of Males,% Widowed,% Never Married,% Married Spouse Present,% Divorced,% Seperated,% Married Spouse Absent,% White,% Black/African American,% Other Race,% Two major Races,% Japanese,% Other Asian,% American Indian or Alaska Native,% Chinese,% Three or More Races,% Citizen,% Non-Citizen,% Naturalized Citizen,% English Is Spoken At Home,% Another Language Is Spoken At Home,% Some College or Bachelor Degree,% High School or Lower Education,% Masters or Professional Certificate,% Doctoral Degree,% Employed,% Not In The Labor Force,% Unemployed,state_po,county_name,democrat,green,liberitarian,other,republican,winner,percent_rep,percent_dem,winning_margin
0,2008,1003,27222.4,902.2,48.6,53308.6,94.2,5.8,52.5,47.5,8.3,12.1,63.4,13.2,1.3,1.6,91.1,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,97.1,1.3,1.6,97.1,2.9,51.8,39.3,8.4,0.5,54.1,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0,75.259479,23.811922,51.447558
1,2008,1015,20951.6,551.7,47.1,39712.5,89.1,10.9,54.8,45.2,8.7,19.5,54.5,11.5,3.0,2.8,80.5,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,97.3,1.5,1.2,95.4,4.6,41.1,52.6,5.0,1.2,54.3,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0,65.691889,33.170870,32.521019


In [18]:
pres_df.describe()

,Year,County FIP Code,Average Annual Income of Individuals,Average Amount Of Mortgage Payments,Average Age,Average Annual Total Family Income,% of Individuals Who Did Not Receive Foodstamps,% of Individuals Who Received Foodstamps,% of Females,% of Males,% Widowed,% Never Married,% Married Spouse Present,% Divorced,% Seperated,% Married Spouse Absent,% White,% Black/African American,% Other Race,% Two major Races,% Japanese,% Other Asian,% American Indian or Alaska Native,% Chinese,% Three or More Races,% Citizen,% Non-Citizen,% Naturalized Citizen,% English Is Spoken At Home,% Another Language Is Spoken At Home,% Some College or Bachelor Degree,% High School or Lower Education,% Masters or Professional Certificate,% Doctoral Degree,% Employed,% Not In The Labor Force,% Unemployed,democrat,green,liberitarian,other,republican,winner,percent_rep,percent_dem,winning_margin
count,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000,1670.000000,1670.000000,1.670000e+03,1670.000000,1670.000000,1670.000000,1670.000000
mean,2014.194012,28717.220958,28841.084850,906.672635,46.903772,58749.243473,90.251257,9.748743,51.855629,48.144371,6.777066,25.837186,52.495150,11.105329,1.721317,2.062275,78.738683,9.917784,2.978862,3.136048,0.235689,3.035150,0.759401,0.928862,0.272455,88.969461,5.269042,5.761198,85.102335,14.897665,49.909222,38.882335,9.860778,1.347904,58.226347,38.028922,3.745689,1.158984e+05,167.460479,668.785629,4803.562275,8.568574e+04,0.488623,48.252009,48.924939,-0.672930
std,4.410969,15595.725527,7295.609512,291.128525,2.787388,14606.621299,4.952513,4.952513,1.830548,1.830548,1.559419,7.176813,7.509596,2.044045,0.766393,0.795502,14.646107,10.614890,3.912003,3.870875,1.090427,3.688328,2.406652,1.840668,0.680919,9.457760,4.631289,5.320845,13.393902,13.393902,5.909867,9.208955,4.384230,0.979554,6.558229,6.335632,1.373302,1.918876e+05,805.370013,2083.527700,10236.284021,9.659596e+04,0.500020,14.854136,14.849056,29.587523
min,2008.000000,1003.000000,14897.300000,421.100000,36.700000,34914.700000,65.600000,0.900000,33.400000,42.000000,2.500000,11.600000,26.800000,5.600000,0.000000,0.500000,18.600000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,44.800000,0.000000,0.000000,6.900000,1.400000,25.600000,13.000000,2.300000,0.000000,30.700000,20.800000,0.800000,2.474000e+03,0.000000,0.000000,0.000000,2.187000e+03,0.000000,4.087474,9.724106,-86.776347
25%,2012.000000,13135.000000,23894.650000,698.025000,45.200000,48357.400000,87.300000,6.100000,51.000000,47.100000,5.700000,20.900000,48.000000,9.625000,1.200000,1.500000,71.425000,2.625000,0.725000,1.100000,0.000000,1.000000,0.200000,0.200000,0.100000,86.400000,2.125000,2.200000,81.400000,6.200000,46.100000,32.300000,6.700000,0.700000,54.300000,33.700000,2.800000,2.843275e+04,0.000000,0.000000,756.250000,3.562700e+04,0.000000,39.189329,38.168029,-18.749241
50%,2016.000000,29099.000000,27351.850000,828.550000,46.900000,55458.800000,90.900000,9.100000,52.000000,48.000000,6.700000,24.900000,52.800000,11.100000,1.600000,1.950000,82.600000,6.400000,1.600000,1.700000,0.100000,1.900000,0.300000,0.400000,0.200000,92.200000,3.600000,3.900000,89.900000,10.100000,50.000000,38.800000,9.100000,1.100000,58.700000,37.500000,3.500000,5.305350e+04,0.000000,0.000000,1856.000000,5.448950e+04,0.000000,48.062352,48.838833,-0.756352
75%,2020.000000,42018.500000,32176.200000,1039.600000,48.400000,66187.150000,93.900000,12.700000,52.900000,49.000000,7.700000,29.400000,57.600000,12.400000,2.200000,2.500000,89.700000,13.300000,3.600000,3.700000,0.200000,3.600000,0.700000,0.900000,0.3

[Princeton defines "competitiveness zone" as districts where the Democratic and Republican vote share is between 46.5%-53.5%.](https://gerrymander.princeton.edu/redistricting-report-card-methodology) We will use a slightly expanded definition.

In [19]:
def rate_county(df, col):
    if col >= 12:
        return 'Safe R'
    if 5 < col < 12:
        return 'Lean R'
    if -5 <= col <= 5:
        return 'Swing'
    if -12 < col < -5:
        return 'Lean D'
    else:
        return 'Safe D'

In [20]:
pres_df['rating'] = pres_df['winning_margin'].apply(lambda x: rate_county(pres_df, x))

In [21]:
pres_df['rating'].value_counts()

Safe R    587
Safe D    572
Swing     216
Lean D    168
Lean R    127
Name: rating, dtype: int64

In [22]:
us_census_divisions = {
    'New England': ['CT', 'ME', 'MA', 'NH', 'RI', 'VT'],
    'Mid-Atlantic': ['NJ', 'NY', 'PA'],
    'East North Central': ['IL', 'IN', 'MI', 'OH', 'WI'],
    'West North Central': ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'],
    'South Atlantic': ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'WV'],
    'East South Central': ['AL', 'KY', 'MS', 'TN'],
    'West South Central': ['AR', 'LA', 'OK', 'TX'],
    'Mountain': ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY'],
    'Pacific': ['AK', 'CA', 'HI', 'OR', 'WA']
}


In [23]:
def region_assignment(dic, val):
    for k, value in dic.items():
        if val in value:
            return k
        else:
            pass

In [24]:
region_assignment(us_census_divisions, 'NM')

'Mountain'

In [25]:
pres_df['census_region'] = pres_df['state_po'].apply(lambda x: region_assignment(us_census_divisions, x))

In [26]:
pres_df.head()

,Year,County FIP Code,Average Annual Income of Individuals,Average Amount Of Mortgage Payments,Average Age,Average Annual Total Family Income,% of Individuals Who Did Not Receive Foodstamps,% of Individuals Who Received Foodstamps,% of Females,% of Males,% Widowed,% Never Married,% Married Spouse Present,% Divorced,% Seperated,% Married Spouse Absent,% White,% Black/African American,% Other Race,% Two major Races,% Japanese,% Other Asian,% American Indian or Alaska Native,% Chinese,% Three or More Races,% Citizen,% Non-Citizen,% Naturalized Citizen,% English Is Spoken At Home,% Another Language Is Spoken At Home,% Some College or Bachelor Degree,% High School or Lower Education,% Masters or Professional Certificate,% Doctoral Degree,% Employed,% Not In The Labor Force,% Unemployed,state_po,county_name,democrat,green,liberitarian,other,republican,winner,percent_rep,percent_dem,winning_margin,rating,census_region
0,2008,1003,27222.4,902.2,48.6,53308.6,94.2,5.8,52.5,47.5,8.3,12.1,63.4,13.2,1.3,1.6,91.1,6.5,0.4,0.5,0.0,0.6,0.5,0.1,0.2,97.1,1.3,1.6,97.1,2.9,51.8,39.3,8.4,0.5,54.1,43.5,2.4,AL,BALDWIN,19386.0,0.0,0.0,756.0,61271.0,1.0,75.259479,23.811922,51.447558,Safe R,East South Central
1,2008,1015,20951.6,551.7,47.1,39712.5,89.1,10.9,54.8,45.2,8.7,19.5,54.5,11.5,3.0,2.8,80.5,17.1,0.1,0.9,0.1,0.7,0.3,0.2,0.0,97.3,1.5,1.2,95.4,4.6,41.1,52.6,5.0,1.2,54.3,41.1,4.5,AL,CALHOUN,16334.0,0.0,0.0,560.0,32348.0,1.0,65.691889,33.170870,32.521019,Safe R,East South Central
2,2008,1055,24017.6,658.6,48.2,47650.0,94.2,5.8,54.6,45.4,10.5,13.8,60.9,11.7,1.5,1.6,85.5,11.6,0.0,1.6,0.0,0.7,0.2,0.1,0.2,98.6,0.9,0.5,98.0,2.0,45.3,46.6,7.9,0.2,50.3,46.8,2.9,AL,ETOWAH,13497.0,0.0,0.0,645.0,30595.0,1.0,68.388582,30.169658,38.218924,Safe R,East South Central
3,2008,1073,28109.5,812.4,47.2,54993.2,93.5,6.5,55.0,45.0,9.5,23.5,50.3,12.3,2.5,1.9,63.1,35.1,0.4,0.7,0.0,0.5,0.1,0.1,0.0,97.5,1.3,1.2,96.6,3.4,47.8,42.0,9.3,0.9,58.5,38.1,3.4,AL,JEFFERSON,166121.0,0.0,0.0,2482.0,149921.0,0.0,47.067411,52.153370,-5.085959,Lean D,East South Central
4,2008,1081,24870.4,725.1,40.6,45575.0,92.4,7.6,52.1,47.9,4.9,32.1,51.2,8.3,1.7,2.0,75.2,21.0,0.3,0.6,0.1,1.7,0.3,0.8,0.2,94.4,4.1,1.5,93.2,6.8,53.3,32.2,10.8,3.7,59.8,36.3,3.8,AL,LEE,21498.0,0.0,0.0,597.0,32230.0,1.0,59.328118,39.572941,19.755177,Safe R,East South Central


In [27]:
pres_df['census_region'].value_counts()

South Atlantic        380
East North Central    310
Mid-Atlantic          235
Pacific               213
West South Central    194
West North Central    106
East South Central     94
Mountain               70
New England            64
Name: census_region, dtype: int64

In [28]:
# pres_df.drop(['County FIP Code', 'county_name', 'democrat', 'green', 'liberitarian', 'other', 'republican'], inplace = True, axis = 1)